# AE2 - Assignment on Panel Data (2025)

| Nr|**Name**|**Student ID**|
|---|--------|--------------|
|1.|Olga Milovanova|12879436| 
|2.|Alessandro Bonetti|15855902| 
|3.|Bowen Ma|12960780|  

# Code
This section contains the code needed to answer the 4 questions of the assignment's Analysis and produce the related results.

The packages needed to run the code are:
* NumPy
* Pandas
* Statsmodels
* Matplotlib.pyplot
* SciPy
* Parallel, from joblib

In [22]:
# --- PACKAGES NEEDED ---

import numpy as np
import pandas as pd
import statsmodels as sm
import matplotlib.pyplot as plt
import scipy as sc
from joblib import Parallel, delayed # Parallel running to reduce running time of the code


In [23]:
# --- FUNCTIONS ---

## Function for vec operator
def vec(mA):
    return np.asmatrix(mA.ravel('F')).T # ravel flattens the inpute array into 1-dim
                                      # F specifies flatten by columns
                                      # asmatrix convert the input into a numpy matrix

## Function for time-series or cross-sectional de-meaning
def Demean(mA, iAxis):

        if iAxis == 0:
            # when iAxis=0 substract mean alnong column
            mA = mA - np.mean(mA,axis=iAxis).reshape(1,int(np.shape(mA)[1-iAxis]))
        else:
            # when iAxis=0 substract mean alnong row
            mA = mA - np.mean(mA,axis=iAxis).reshape(int(np.shape(mA)[1-iAxis]),1)

        return mA

## Function to generate the data
def GenrData (iSizeT, iSizeN, dAlpha, iS):  # set iS =1 when calling Number of "lagged" periods to skip or warm up before considering valid data.
    
    mErrors  = np.random.randn(iSizeN, iSizeT+iS+1)
    mDataY = np.zeros((iSizeN,iSizeT+iS+1))
    for t in range(1,iSizeT+iS+1):
        mDataY[:,[t]] = dAlpha*mDataY[:,[t-1]] +  mErrors[:,[t]]
        
    return (np.transpose(mDataY[:,iS+1:]), np.transpose(mDataY[:,iS:iSizeT+iS]))# dim (iSizeT, iSizeN)

## Function for the FE estimator
def FE(y,x):
    '''
    The function to calculate the fixed effect estimator
    First we  demean the regressor and the dependent variable
    Then we stack the variables to regress using OLS
    '''
    # First demean the variables
    y_demeaned=vec(Demean(y, 0))
    x_demeaned=vec(Demean(x, 0))

    # Compute the FE estimator
    alpha_hat = np.linalg.inv(x_demeaned.T @ x_demeaned) @ x_demeaned.T @ y_demeaned
    # returns a (1,1) matrix

    # Convert it to a scalar
    alpha_hat = alpha_hat.item()

    return alpha_hat

## Function for the HPJ estimator
def HPJ (mDataY, mDataX, dEstimator, Estimation): # input is dEstimator(FE_estimator) and  Estimation(FE function)

    iSizeT,iSizeN = np.shape(mDataY)
    iHalf = int(np.floor(iSizeT/2))

    dEstimator1 = Estimation (mDataY[:iHalf,:], mDataX[:iHalf,:])
    dEstimator2 = Estimation (mDataY[iHalf:,:], mDataX[iHalf:,:])
    dHPJ = 2*dEstimator-0.5*(dEstimator1+dEstimator2)

    return (dHPJ,dEstimator1,dEstimator2)

## Function to generate the bootstrapped data
def GenrDataBootstrap (iSizeT, iSizeN, dAlpha_bootstrap, vY0, mResiduals):

    # Initialize matrix of observations
    mDataY_B = np.zeros((iSizeN,iSizeT+1))

    # Transpose mErrors_B and vY0 to guarantee data compatibility
    mResiduals = np.transpose(mResiduals)
    vY0 = np.transpose(vY0)

    # Generate Rademacher weights
    mWeights = np.random.choice([1, -1], size=(iSizeN, iSizeT))
    # Generate bootstrapped residuals
    mErrors_B = np.multiply(mResiduals, mWeights) # element-wise product

    # Set initial condition on Y
    mDataY_B[:,0] = vY0

    for t in range(1, iSizeT+1):
      # Generate Y
      mDataY_B[:,[t]] = dAlpha_bootstrap * mDataY_B[:,[t-1]] + mErrors_B[:,[t-1]]

    mDataY_lag_B = np.transpose(mDataY_B[:,:iSizeT])
    mDataY_B = np.transpose(mDataY_B[:,1:])

    return mDataY_B, mDataY_lag_B

## Function for bootstrap
def BootstrapMC (mResiduals, vY0, iB, dLevel, vNull, vEst, dAlpha_bootstrap, chosen_est):  # 

    iSizeT, iSizeN = np.shape(mResiduals)       # extract T and N
    iSizeP = np.shape(vNull)[0]                 # number of different hypotheses to be tested
    iSizeEst = len(vEst)                        # number of estimates to be tested

    iUnitLow = int(np.floor((dLevel/2)*iB))     # define lower quantile unit
    iUnitHigh = int(np.floor((1-dLevel/2)*iB))  # define upper quantile unit

    mResB = np.zeros((iB,iSizeEst))             # array to store bootstrapped estimates
    mResB[0,:] = vEst                           # include original estimate while constructing CI

    # Compute bootstrapped estimates
    for b in range(1,iB):

        # Generate bootstrapped data
        mDataY_B, mDataY_lag_B = GenrDataBootstrap (iSizeT, iSizeN, dAlpha_bootstrap, vY0, mResiduals)

        dFE_B = FE (mDataY_B, mDataY_lag_B) # bootstrapped FE estimates
        mResB[b,0] = dFE_B - dAlpha_bootstrap # center estimates around the value of alpha used in the bootstrap and store them

        if chosen_est == 'HPJ':          
          dFE_HPJ_B = HPJ (mDataY_B, mDataY_lag_B, dFE_B, FE)[0] # bootstrapped HPJ estimates
          mResB[b,0] = dFE_HPJ_B - dAlpha_bootstrap # center estimates around the value of alpha used in the bootstrap and store them

    # Sort bootstrapped estimates
    mResB_Sorted = np.sort(mResB, axis=0)

    # Initialize array to store rejections
    mReject = np.zeros((iSizeEst, iSizeP)) # dimension is:
                                # number of estimators to be tested
                                # multiplied by number of hypotheses to be tested

    # Run the test
    for k in range(0, iSizeEst):
        dLowQ = mResB_Sorted[iUnitLow, k]     # compute lower quantile
        dHighQ = mResB_Sorted[iUnitHigh, k]   # compute upper quantile
        dLow = dLowQ
        dHigh = dHighQ

        # Run and store test result
        mReject[k,:] = np.where((vNull < dHigh) & (vNull > dLow), 0, 1)

    return mReject.T

## Function for the Monte Carlo study
def MC_study (iSizeT, dKappa, dGamma, iM, iB):

    np.random.seed(42)
    
    # To store estimation results
    mResultsEstimation = np.asmatrix(np.zeros((iM,2)))
    
    # To store testing results for any choice of P. Standard P=1
    iP = 1
    mResultsTesting = np.zeros((iM,3*iP))

    for m in range (0,iM):

        # True value is a function of T,\gamma,\kappa
        dAlpha_0 =  np.exp(-iSizeT**-dGamma)

        iSizeN = int(iSizeT**(1/dKappa)) 

        mDataY, mDataY_lag = GenrData (iSizeT, iSizeN, dAlpha_0,iS) # generating the data

        dFE = FE(mDataY, mDataY_lag)              # FE estimator
        dFE_HPJ = HPJ(mDataY, mDataY_lag,dFE,FE) 
        dHPJ = dFE_HPJ[0]                         # HPJ estimator

        mResultsEstimation[m,0] = dFE - dAlpha_0
        mResultsEstimation[m,1] = dHPJ - dAlpha_0

        # FE bootstrap initialization
        vEst_FE = np.array([dFE])
        dAlpha_bootstrap_FE = dFE
        mResiduals_FE = mDataY - dFE * mDataY_lag
        vNull_FE = np.array([dFE - dAlpha_0])              

        # HPJ bootstrap initialization
        vEst_HPJ = np.array([dHPJ])
        dAlpha_bootstrap_HPJ = dHPJ
        mResiduals_HPJ = mDataY - dHPJ * mDataY_lag
        vNull_HPJ = np.array([dHPJ - dAlpha_0])

        # Q4 bootstrap initialization
        dAlpha_bootstrap = dAlpha_0
        
        vY0 = mDataY_lag[0,:]  # initial condition for bootstrap
        dLevel = 0.05          # significance level

        # Run bootstrap
        mRejectFE = BootstrapMC (mResiduals_FE, vY0, iB, dLevel, vNull_FE, vEst_FE, dAlpha_bootstrap_FE, "FE")
        mRejectHPJ = BootstrapMC (mResiduals_HPJ, vY0, iB, dLevel, vNull_HPJ, vEst_HPJ, dAlpha_bootstrap_HPJ, "HPJ")
        mRejectQ4 = BootstrapMC (mResiduals_FE, vY0, iB, dLevel, vNull_FE, vEst_FE, dAlpha_bootstrap, "FE") # Only change dAlpha_bootstrap

        # Store testing results
        mResultsTesting[m,0] = mRejectFE[:,-1]  
        mResultsTesting[m,1] = mRejectHPJ[:,-1]
        mResultsTesting[m,2] = mRejectQ4[:,-1]
        
    # Bias for FE 
    vBias_FE = np.mean(mResultsEstimation[:, 0])

    # Bias for HPJ 
    vBias_HPJ = np.mean(mResultsEstimation[:, 1])
    
    # RMSE
    vRMSE = np.sqrt(np.mean(np.power(mResultsEstimation,2),axis=0))

    # Rejection frequencies
    mFreqFE = np.mean(mResultsTesting[:,0])
    mFreqHPJ = np.mean(mResultsTesting[:,1])
    mFreqQ4 = np.mean(mResultsTesting[:,2]) 

    print("Iteration complete for:","T=",iSizeT,", N=",iSizeN,", alpha=",np.round(dAlpha_0, 3)) 

    return vBias_FE,vBias_HPJ, vRMSE, mFreqFE, mFreqHPJ, mFreqQ4, dAlpha_0, iSizeN


In [42]:
# --- OBTAINING RESULTS ---

# Wrapper to handle a single simulation
def run_simulation(T, kappa, gamma, iM, iB, iS):
    """
    Run a single Monte Carlo study and return results for the given parameters.
    """
    vBias_FE, vBias_HPJ, vRMSE, mFreqFE, mFreqHPJ, mFreq, dAlpha_0, iSizeN = MC_study(
        T, kappa, gamma, iM, iB
    )
    return {
        "T": T,
        "N": iSizeN,
        "κ": kappa,
        "γ": gamma,
        "α": dAlpha_0,
        "RMSE for FE estimator": vRMSE[0, 0],  # RMSE for FE
        "RMSE for HPJ estimator": vRMSE[0, 1],  # RMSE for HPJ
        "MC Bias for FE estimator": vBias_FE,
        "MC Bias for HPJ estimator": vBias_HPJ,
        "Rejection freq. FE": mFreqFE,
        "Rejection freq. HPJ": mFreqHPJ,
        "Rejection freq. Q4": mFreq,
    }

# Parameters
T_values = [20, 50, 100]
kappa_values = [0.6, 0.8, 1.0]
gamma_values = [0.75, 1.0, 1.25]
iM = 4000
iB = 199
iS = 1

# Run simulations in parallel
results = Parallel(n_jobs=-1, verbose=10)(
    delayed(run_simulation)(T, kappa, gamma, iM, iB, iS)
    for T in T_values
    for kappa in kappa_values
    for gamma in gamma_values
)

# Create DataFrame directly from list of dictionaries
df = pd.DataFrame(results)
df = df.round(4).reset_index(drop=True)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed: 63.3min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed: 117.6min
[Parallel(n_jobs=-1)]: Done  15 out of  27 | elapsed: 254.1min remaining: 203.3min
[Parallel(n_jobs=-1)]: Done  18 out of  27 | elapsed: 297.7min remaining: 148.8min
[Parallel(n_jobs=-1)]: Done  21 out of  27 | elapsed: 534.3min remaining: 152.7min
[Parallel(n_jobs=-1)]: Done  24 out of  27 | elapsed: 641.9min remaining: 80.2min
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed: 1575.3min finished


In [46]:
### Results 
df

,T,N,κ,γ,α,RMSE for FE estimator,RMSE for HPJ estimator,MC Bias for FE estimator,MC Bias for HPJ estimator,Rejection freq. FE,Rejection freq. HPJ,Rejection freq. Q4
0,20,147,0.6,0.75,0.8997,0.1373,0.0256,-0.1365,-0.0118,0.4142,0.1003,0.0465
1,20,147,0.6,1.00,0.9512,0.1488,0.0313,-0.1482,-0.0221,0.5508,0.0882,0.0442
2,20,147,0.6,1.25,0.9766,0.1498,0.0300,-0.1493,-0.0213,0.4812,0.0480,0.0532
3,20,42,0.8,0.75,0.8997,0.1402,0.0440,-0.1377,-0.0118,0.1342,0.0945,0.0458
4,20,42,0.8,1.00,0.9512,0.1518,0.0473,-0.1497,-0.0226,0.1730,0.0758,0.0468
5,20,42,0.8,1.25,0.9766,0.1528,0.0457,-0.1509,-0.0221,0.1417,0.0542,0.0420
6,20,20,1.0,0.75,0.8997,0.1465,0.0640,-0.1411,-0.0135,0.0905,0.1030,0.0432
7,20,20,1.0,1.00,0.9512,0.1576,0.0657,-0.1529,-0.0234,0.1030,0.0832,0.0445
8,20,20,1.0,1.25,0.9766,0.1583,0.0634,-0.1540,-0.0222,0.0853,0.0588,0.0430
9,50,678,0.6,0.75,0.9482,0.0557,0.0046,-0.0556,-0.0004,0.7352,0.0980,0.0428


# Discussion

## Bias (Question 1)

### Fixed effects estimator (FE)

- In virtually every scenario the bias for the FE estimator is negative, which aligns with established literature. 

- For $T$ smaller (i.e., $T=20$), the bias is larger with respect to the other scenarios. 

    We can think of this situation as a setting with T fixed. The reason for the larger bias is the so called "Nickell bias" (Nickell, 1981): for T fixed, the asymptotic result is that the FE estimator is inconsistent. 
    
    This distortion arises because the first difference transformation (used to get rid of the fixed effects) introduces into the panel data model a correlation between the (transformed) lagged dependent variable and the (transformed) error term. This is known as the *incidental parameters problem*.

- The bias diminishes as $T$ increases to $50$, and it becomes even smaller for $T=100$, which reflects improved consistency. 

    This confirms the asymptotic result for a setting with $T$ large (and $N$ large), where the FE estimator becomes consistent. Analytically, we can write: $E(\hat{a}_{FE}) - \alpha_0 \xrightarrow{p} -\frac{1}{T} (1+\alpha_0)$, which goes to $0$ as $T$ increases.
    
    Howevever, for T large, the FE estimator still displays an asymptotic bias which can make standard inference unreliable (Hahn and Kuersteiner, 2002). This becomes evident if we consider the expression:

    $\sqrt{N T} (\hat{a}_{FE} - \alpha_0) \xrightarrow{d} \mathcal{N}(-k(1+\alpha_0), (1-\alpha_0^2))$

    where $N, T \to \infty, \quad N/T \to k^2 \in [0, \infty)$.

    The asymptotic bias $-k(1+\alpha_0)$ is negligible only if $N/T \approx 0$, i.e., when the ratio between $N$ and $T$ is very small in the sample at hand. This is clearly not the case for some of our parameter combinations. 
    
    For example, let us consider the scenario(s) where $T=100$. Here, the FE estimate is reliable across the different values of $N$, as the bias hovers around $3\%$ of the true value of $\alpha$. However, if our concern is not only estimation, but also inference, we must pay attention to the ratio $N/T$. For larger values of $N$ (and keeping $T=100$), $k$ in the previous expression becomes larger, making the asymptotic bias bigger and inference results less reliable.

    We can obtain the approximate value of the asymptotic bias $-k(1+\alpha_0)$ by multiplying by $\sqrt{N T}$ the FE bias (contained in the column *MC Bias for FE estimator* of the table at the end of the report).

### Half Panel Jackknife estimator (HPJ)

- The bias for the HPJ estimator is significantly smaller compared to the FE estimator. 
- Indeed, the HPJ estimator is designed to deal with the Nickell Bias by explicitly estimating and subtracting the bias through a Jackknife-based method. 
- The bias of the HPJ estimator almost always takes negative values (although very small). As the bias for the FE estimator is usually negative, this could possibly signal that in our data the HPJ procedure falls slightly short of consistently offsetting the FE bias.

## RMSE (Question 1)

### FE estimator

- RMSE values for the FE estimator generally decline as $T$ increases.

- They are highest when $T$ is small and $\gamma$ is large. Indeed, a larger value of $\gamma$ is associated to a true value of $\alpha$ closer to 1 (unit root), which in turn implies a more persistent AR(1) process and noisier estimates.

### HPJ estimator

- RMSE is considerably lower than FE’s under all parameter scenarios, particularly as $T$ grows. 

- This shows the effectiveness of Jackknife-based bias correction in dealing with the Nickell bias. 

- We also observe that results are robust to values of $\alpha \approx 1$ (very close to unit root). This shows the practical superiority of the Jackknife bias correction with respect to analytical approaches like the one illustrated by Hahn and Kuersteiner (2002, p.1645), where "the bias correction for the stationary case is not expected to work under the unit root".

## Rejection Frequencies

### FE rejection frequencies (Question 2)
- FE rejection frequencies can be extremely high when the ratio $N/T$ is high. This indicates that the confidence intervals are undercovering the true parameter value (over-rejection of the null).
    
    For example, rejection frequencies approach 1 when $N=2154$ and $T=100$, so that $N/T=21.54$.
    
    Instead, if we consider $N=100$ and $T=100$ (i.e. $N/T=1$), rejection frequencies become much lower: between $10\%$ and $20\%$.

    Let us consider parameter combinations with the same ratio $N/T=1$. Rejection frequencies for $(N=50, T=50)$, and for $(N=20, T=20)$ are slightly lower than for $(N=100, T=100)$, but they are of a comparable order of magnitude, as they hover around $10\%$. 
    
- This shows that what matters for inference based on the FE estimator is the ratio between $N$ and $T$, not simply the value of $T$.

- This result confirms our previous point about the unreliability of inference based on the FE estimator when $N/T$ is sizeable. Indeed, if $N/T$ is not small enough, the asymptotic bias $-k(1+\alpha_0)$ is not negligible, which badly distorts inference procedures.

- We also notice that, *ceteris paribus*, values of $\gamma=1$ inflate rejection frequencies.

- According to Gonçalves and Kaffo (2015), the RW bootstrap applied to the (biased) FE estimator should be able to give acceptable results in terms of inference, thanks to the "asymptotic validity of a bootstrap percentile-t interval based on $t_{\hat{\theta}^{*}_{rd}}$ [the bootstrapped t-statistic]" (ibid.). They argue that "the recursive-design bootstrap contains a built-in bias correction term that mimics the incidental parameter bias induced by the individual fixed effects" (p.411). However, this asymptotic property seems not to be valid in our finite-sample environment. Our results indicate that the built-in bias correction term of RWB may not be enough to deal with the FE bias and provide reasonable rejection frequencies.

### HPJ rejection frequencies (Question 3)
- The HPJ rejection rates are generally closer to the nominal $5\%$, but they still can be higher for smaller $T$.

- For larger $T$, they tend closer to the nominal significance rate, and show significantly better results than the FE rejection rates.

- For larger $\gamma$ (=larger $\alpha_0$) the rejection rates decrease significantly.

- Differently from the FE estimator, the ratio $N/T$ does not have influence rejection frequencies. This is due to the fact that the HPJ estimator explicitly corrects for the asymptotic bias $-k(1+\alpha_0)$, neutralizing the effect of a large $N/T$ ratio.

- In general, these results seem to support one of the main conclusions of Gonçalves and Kaffo (2015, p.413), i.e., the validity of a recursive-design bootstrap applied to a bias-corrected fixed effect estimator (the HPJ estimator in our case). 

- However, the inference refinement brought about by the bias-corrected estimator (with respect to the *biased* FE estimator) is much higher than implied by Gonçalves and Kaffo (2015).

### Question 4 Rejection Frequencies
- Setting the bootstrap DGP to the true $\alpha_0$ (rather than $\hat{\alpha}_{FE}$) brings the rejection rates mostly closer to the nominal $5\%$.

- Even for large $T$, these rejection rates stay around $0.04-0.08$, indicating much more stable coverage than relying on the biased FE estimate.

- This result underlines once more the distortion introduced into inference by the asymptotic bias of the FE estimator.

## Copy of final results:

| | **T**  | **N**   | **κ**  | **γ**   | **α**_0| **RMSE for FE estimator** | **RMSE for HPJ estimator** | **MC Bias for FE estimator** | **MC Bias for HPJ estimator** | **Rejection freq. FE** | **Rejection freq. HPJ** | **Rejection freq. Q4** |
|-------:|-------:|--------:|------:|-------:|---------:|--------------------------:|---------------------------:|----------------------------:|-----------------------------:|-----------------------:|------------------------:|------------------------:|
| 0      | 20     | 147     | 0.6   | 0.75    | 0.8997    | 0.1373                    | 0.0256                     | -0.1365                     | -0.0118                      | 0.4142                 | 0.1003                  | 0.0465                  |
| 1      | 20     | 147     | 0.6   | 1.00    | 0.9512    | 0.1488                    | 0.0313                     | -0.1482                     | -0.0221                      | 0.5508                 | 0.0882                  | 0.0442                  |
| 2      | 20     | 147     | 0.6   | 1.25    | 0.9766    | 0.1498                    | 0.0300                     | -0.1493                     | -0.0213                      | 0.4812                 | 0.0480                  | 0.0532                  |
| 3      | 20     | 42      | 0.8   | 0.75    | 0.8997    | 0.1402                    | 0.0440                     | -0.1377                     | -0.0118                      | 0.1342                 | 0.0945                  | 0.0458                  |
| 4      | 20     | 42      | 0.8   | 1.00    | 0.9512    | 0.1518                    | 0.0473                     | -0.1497                     | -0.0226                      | 0.1730                 | 0.0758                  | 0.0468                  |
| 5      | 20     | 42      | 0.8   | 1.25    | 0.9766    | 0.1528                    | 0.0457                     | -0.1509                     | -0.0221                      | 0.1417                 | 0.0542                  | 0.0420                  |
| 6      | 20     | 20      | 1.0   | 0.75    | 0.8997    | 0.1465                    | 0.0640                     | -0.1411                     | -0.0135                      | 0.0905                 | 0.1030                  | 0.0432                  |
| 7      | 20     | 20      | 1.0   | 1.00    | 0.9512    | 0.1576                    | 0.0657                     | -0.1529                     | -0.0234                      | 0.1030                 | 0.0832                  | 0.0445                  |
| 8      | 20     | 20      | 1.0   | 1.25    | 0.9766    | 0.1583                    | 0.0634                     | -0.1540                     | -0.0222                      | 0.0853                 | 0.0588                  | 0.0430                  |
| 9      | 50     | 678     | 0.6   | 0.75    | 0.9482    | 0.0557                    | 0.0046                     | -0.0556                     | -0.0004                      | 0.7352                 | 0.0980                  | 0.0428                  |
| 10     | 50     | 678     | 0.6   | 1.00    | 0.9802    | 0.0618                    | 0.0077                     | -0.0618                     | -0.0063                      | 0.9552                 | 0.0808                  | 0.0635                  |
| 11     | 50     | 678     | 0.6   | 1.25    | 0.9925    | 0.0614                    | 0.0066                     | -0.0613                     | -0.0052                      | 0.8588                 | 0.0230                  | 0.0790                  |
| 12     | 50     | 132     | 0.8   | 0.75    | 0.9482    | 0.0563                    | 0.0103                     | -0.0559                     | -0.0004                      | 0.1872                 | 0.0835                  | 0.0488                  |
| 13     | 50     | 132     | 0.8   | 1.00    | 0.9802    | 0.0623                    | 0.0117                     | -0.0620                     | -0.0063                      | 0.3268                 | 0.0740                  | 0.0520                  |
| 14     | 50     | 132     | 0.8   | 1.25    | 0.9925    | 0.0618                    | 0.0106                     | -0.0615                     | -0.0051                      | 0.2198                 | 0.0268                  | 0.0548                  |
| 15     | 50     | 50      | 1.0   | 0.75    | 0.9482    | 0.0573                    | 0.0166                     | -0.0563                     | -0.0004                      | 0.0830                 | 0.0845                  | 0.0392                  |
| 16     | 50     | 50      | 1.0   | 1.00    | 0.9802    | 0.0631                    | 0.0170                     | -0.0623                     | -0.0062                      | 0.1212                 | 0.0645                  | 0.0395                  |
| 17     | 50     | 50      | 1.0   | 1.25    | 0.9925    | 0.0626                    | 0.0158                     | -0.0619                     | -0.0050                      | 0.0905                 | 0.0362                  | 0.0430                  |
| 18     | 100    | 2154    | 0.6   | 0.75    | 0.9689    | 0.0276                    | 0.0017                     | -0.0275                     | 0.0011                       | 0.9540                 | 0.0872                  | 0.0472                  |
| 19     | 100    | 2154    | 0.6   | 1.00    | 0.9900    | 0.0313                    | 0.0030                     | -0.0313                     | -0.0027                      | 1.0000                 | 0.0782                  | 0.0525                  |
| 20     | 100    | 2154    | 0.6   | 1.25    | 0.9968    | 0.0309                    | 0.0022                     | -0.0309                     | -0.0019                      | 0.9975                 | 0.0165                  | 0.0738                  |
| 21     | 100    | 316     | 0.8   | 0.75    | 0.9689    | 0.0277                    | 0.0036                     | -0.0276                     | 0.0011                       | 0.2525                 | 0.0798                  | 0.0450                  |
| 22     | 100    | 316     | 0.8   | 1.00    | 0.9900    | 0.0315                    | 0.0042                     | -0.0314                     | -0.0027                      | 0.6040                 | 0.0693                  | 0.0488                  |
| 23     | 100    | 316     | 0.8   | 1.25    | 0.9968    | 0.0310                    | 0.0036                     | -0.0309                     | -0.0019                      | 0.3733                 | 0.0140                  | 0.0528                  |
| 24     | 100    | 100     | 1.0   | 0.75    | 0.9689    | 0.0282                    | 0.0062                     | -0.0279                     | 0.0009                       | 0.1003                 | 0.0853                  | 0.0485                  |
| 25     | 100    | 100     | 1.0   | 1.00    | 0.9900    | 0.0319                    | 0.0064                     | -0.0317                     | -0.0029                      | 0.2130                 | 0.0650                  | 0.0465                  |
| 26     | 100    | 100     | 1.0   | 1.25    | 0.9968    | 0.0314                    | 0.0058                     | -0.0312                     | -0.0021                      | 0.1220                 | 0.0240                  | 0.0472                  |


# References

Gonçalves, S., & Kaffo, M. (2015). Bootstrap inference for linear dynamic panel data models with individual fixed effects. Journal of Econometrics, 186(2), 407–426. https://doi.org/10.1016/j.jeconom.2015.02.017

Hahn, J., & Kuersteiner, G. (2002). Asymptotically unbiased inference for a dynamic panel model with fixed effects when both n and T are large. Econometrica, 70(4), 1639–1657. https://doi.org/10.1111/1468-0262.00342

Nickell, S. (1981). Biases in dynamic models with fixed effects. Econometrica, 49(6), 1417–1426. https://doi.org/10.2307/1911408